In [19]:
import pandas as pd
import numpy as np
import copy

df = pd.read_csv ( "/data/project/Meningioma/31.Clonality/02.pyclonevi/230419/230419.decomposed-facetcnv.tsv", sep = "\t" )
df_count = np.unique ( df ["cluster_id"], return_counts = True )

for i in range ( len(df_count[0]) ) :
    print (df_count[0][i], int(df_count[1][i] / len (df["sample_id"].unique() )) )


0 24
1 15
2 30


In [16]:
for i in range ( len(df_count[0]) ) :
    print (df_count[0][i], df_count[1][i] / len (df["sample_id"].unique() )

0 48
1 30
2 60


In [6]:
df

,mutation_id,sample_id,ref_counts,alt_counts,normal_cn,major_cn,minor_cn,tumour_content,types,gene,variant_classification,cluster_id
0,chr1_116124115,230419_Dura,378,5,2,1,1,1.00,Mutect2,MAB21L3,missense_variant,0
1,chr1_116124115,230419_Tumor,378,0,2,1,1,0.29,Mutect2,MAB21L3,missense_variant,0
2,chr1_204229027,230419_Dura,233,8,2,1,1,1.00,Mutect2,PLEKHA6,synonymous_variant,0
3,chr1_204229027,230419_Tumor,233,0,2,1,1,0.29,Mutect2,PLEKHA6,synonymous_variant,0
4,chr1_74480652,230419_Dura,174,5,2,1,1,1.00,Mutect2,LRRC53,synonymous_variant,0
...,...,...,...,...,...,...,...,...,...,...,...,...
133,chr19_45616430,230419_Tumor,157,5,2,1,0,0.29,Mutect2,EML2,intron_variant,2
134,chr20_37066790,230419_Dura,104,0,2,1,0,1.00,Mutect2,RBL1,synonymous_variant,2
135,chr20_37066790,230419_Tumor,104,6,2,1,0,0.29,Mutect2,RBL1,synonymous_variant,2
136,chr22_43171854,230419_Dura,64,0,2,1,0,1.00,Mutect2,TTLL12,missense_variant,2


In [12]:
import vcf, pysam, pybedtools

tbx = pysam.TabixFile( "/home/goldpm1/Meningioma/04.mutect/03.vep/220930_Dura.MT2.FMC.HF.RMBLACK.vep.vcf.gz" )

a = pybedtools.BedTool('/home/goldpm1/Meningioma/04.mutect/03.vep/220930_multiple.MT2.FMC.HF.RMBLACK.vep.vcf')
b = pybedtools.BedTool("/home/goldpm1/Meningioma/04.mutect/03.vep/220930_Dura.MT2.FMC.HF.RMBLACK.vep.vcf")

ab = a.intersect(b, v = True) 
for interval in ab:
    print ( interval[0:5], sep = "\t" )

# if not tbx.fetch("chr12", 57730658, 57730659) :
#     print ("hello")

['chr1', '150487535', '.', 'A', 'G']
['chr3', '50194058', '.', 'C', 'A']
['chr3', '52322739', '.', 'C', 'T']
['chr5', '103108723', '.', 'A', 'G']
['chr6', '32554712', '.', 'C', 'T']
['chr12', '57730658', '.', 'C', 'T']
['chr12', '106860433', '.', 'T', 'G']
['chr12', '131165351', '.', 'G', 'A']
['chr13', '28438321', '.', 'A', 'G']
['chr13', '113623173', '.', 'G', 'A']
['chr15', '43876007', '.', 'C', 'T']
['chr15', '48332213', '.', 'T', 'G']
['chr15', '78484821', '.', 'T', 'TA']
['chr16', '2235405', '.', 'G', 'A']
['chr17', '7254728', '.', 'T', 'C']
['chr17', '76478828', '.', 'T', 'C']
['chr19', '55341955', '.', 'A', 'G']
['chr21', '25741676', '.', 'GT', 'G']


## Sequenza → PycloneVI

In [ ]:
import pandas as pd
import pybedtools
import pysam
import argparse
import random

SEQUENZA_MUTATION_PATH="/home/goldpm1/Meningioma/11.cnv/2.sequenza/220930_Dura_mutations.txt"
SEQUENZA_SEGMENT_PATH="/home/goldpm1/Meningioma/11.cnv/2.sequenza/220930_Dura_segments.txt"
SEQUENZA_PLOIDY_PATH="/home/goldpm1/Meningioma/11.cnv/2.sequenza/220930_Dura_confints_CP.txt"
SEQUENZA_TO_PYCLONEVI_MATRIX_PATH="/home/goldpm1/Meningioma/31.Clonality/01.make_matrix/220930/220930.sequenza_to_pyclonevi.tsv"
MUTECT_OUTPUT_PATH="/home/goldpm1/Meningioma/04.mutect/02.PASS/220930_Dura.MT2.FMC.HF.vcf"
MUTECT_OUTPUT_PATH="/home/goldpm1/Meningioma/04.mutect/03.vep/220930_Dura.MT2.FMC.HF.RMBLACK.vep.vcf"
GVCF_OUTPUT_PATH="/home/goldpm1/Meningioma/05.gvcf/02.remove_nonref/220930/220930_Dura.g.vcf.gz"
HC_OUTPUT_PATH="/home/goldpm1/Meningioma/06.hc/04.vep/220930/Dura/220930_Dura.DP100.vep.vcf"
HC_BLOOD_RANDOM_PICK_PATH="/home/goldpm1/Meningioma/31.Clonality/01.make_matrix/220930/220930.HC.random_pick_50.bed"
TISSUE="Dura"
Sample_ID="220930"

tbx = pysam.TabixFile (GVCF_OUTPUT_PATH)

SAMPLE_ID = Sample_ID + "_" + TISSUE

ploidy_df = pd.read_csv (SEQUENZA_PLOIDY_PATH, sep = "\t")
TUMOR_PURITY = ploidy_df.iloc[0]["cellularity"]

mutation_df = pd.read_csv (SEQUENZA_MUTATION_PATH, sep = "\t")  
segment_df = pd.read_csv (SEQUENZA_SEGMENT_PATH, sep = "\t")

mutation_df["end"] = mutation_df["position"]
mutation_df["start"] = mutation_df["position"] - 1
mutation_df = mutation_df.reindex (columns = ["chromosome", "start", "end", "GC.percent", "good.reads", "adjusted.ratio", "F", "mutation", "CNn", "CNt", "Mt"] )
mutation_df.head()


######## Mutect call을 골라줌 ###################
mutect_pybed_object = pybedtools.BedTool(MUTECT_OUTPUT_PATH)
segment_pybed_object =  pybedtools.BedTool.from_dataframe(segment_df)

ab = segment_pybed_object.intersect(mutect_pybed_object, wb = True)

for index, contents in enumerate( ab[0] ):
    if "GT:" in contents:
        parsing_sample_index = index 
        break

output_file = open(SEQUENZA_TO_PYCLONEVI_MATRIX_PATH, "w")

for interval in ab:
    CHR, POS = interval[0], interval[2]
    line_dict = {}

    line_dict ["mutation_id"] = str(CHR) + "_"  + str(POS)
    line_dict ["sample_id"] = SAMPLE_ID
    line_dict ["ref_counts"] = str( interval[parsing_sample_index + 2].split(":")[1].split(",")[0])
    line_dict ["alt_counts"] = str( interval[parsing_sample_index + 2].split(":")[1].split(",")[1])
    line_dict["normal_cn"] = str ( 2 )
    line_dict["major_cn"] = str ( interval[10] )
    line_dict["minor_cn"] = str ( interval[11] )
    line_dict["tumour_content"] = str( TUMOR_PURITY  )
    line_dict["type"] = "Mutect2"
    line_dict["gene"] = str( interval [parsing_sample_index - 1] ).split(";")[-1].split("|")[3]
    line_dict["Variant_Classification"] = str( interval [parsing_sample_index - 1] ).split(";")[-1].split("|")[1]
    
    print ( '\t'.join ( list( line_dict.values()) ) )
    print ( '\t'.join ( list( line_dict.values()) ), file = output_file )
        
 

chr1_2787605	220930_Dura	190	5	2	1	1	0.3	Mutect2	TTC34	missense_variant
chr1_42154829	220930_Dura	186	5	2	1	1	0.3	Mutect2	GUCA2B	synonymous_variant
chr1_44802872	220930_Dura	98	14	2	1	1	0.3	Mutect2	TCTEX1D4	downstream_gene_variant
chr1_243221776	220930_Dura	492	48	2	1	1	0.3	Mutect2	CEP170	missense_variant
chr2_3644016	220930_Dura	830	63	2	1	1	0.3	Mutect2	COLEC11	synonymous_variant
chr2_44706287	220930_Dura	278	16	2	1	1	0.3	Mutect2	CAMKMT	missense_variant&splice_region_variant
chr2_178554137	220930_Dura	603	19	2	1	1	0.3	Mutect2	TTN	synonymous_variant
chr2_185808060	220930_Dura	113	5	2	1	1	0.3	Mutect2	FSIP2	intron_variant&NMD_transcript_variant
chr2_222909077	220930_Dura	85	5	2	1	1	0.3	Mutect2	ACSL3	missense_variant
chr2_241240049	220930_Dura	187	5	2	1	1	0.3	Mutect2	HDLBP	missense_variant
chr3_11817144	220930_Dura	363	20	2	1	1	0.3	Mutect2	TAMM41	intron_variant
chr3_49721007	220930_Dura	397	25	2	1	1	0.3	Mutect2	GMPPB	3_prime_UTR_variant
chr5_71656712	220930_Dura	172	8	2	1	1	0.3	Mutect2	MC

In [ ]:
   
    
######## HC call을 골라줌 ###################
hc_blood_pybed_object = pybedtools.BedTool ( HC_BLOOD_RANDOM_PICK_PATH )  
hc_pybed_object = pybedtools.BedTool(HC_OUTPUT_PATH)  #VCF

hc_selected_pybed_object = hc_blood_pybed_object.intersect(hc_pybed_object, wa = True, wb = True)       # Dura/Tumor에서 발견된 HC call 중에서 선택된 50개만 고름 (concordance가 95%이기 때문에 47개정도 예상)


ab = hc_selected_pybed_object.intersect(segment_pybed_object, wa = True, wb = True)
#print (ab [0])

for index, contents in enumerate( ab[0] ):
    if "GT:" in contents:
        parsing_sample_index = index 
        break


for interval in ab:
    CHR, POS = interval[0], int (interval[2]) - 1
    line_dict = {}

    line_dict ["mutation_id"] = str(CHR) + "_"  + str(POS)
    line_dict ["sample_id"] = SAMPLE_ID
    line_dict ["ref_counts"] = str( interval [parsing_sample_index + 1] ).split(":")[1].split(",")[0]
    line_dict ["alt_counts"] = str( interval [parsing_sample_index + 1] ).split(":")[1].split(",")[1]
    line_dict["normal_cn"] = str ( 2 )
    line_dict["major_cn"] = str ( interval[ -3 ] )
    line_dict["minor_cn"] = str ( interval[ -2 ] )
    line_dict["tumour_content"] = str( TUMOR_PURITY  )
    line_dict["type"] = "HC"
    line_dict["gene"] = str( interval [parsing_sample_index - 1] ).split(";")[-1].split("|")[3]
    line_dict["Variant_Classification"] = str( interval [parsing_sample_index - 1] ).split(";")[-1].split("|")[1]
    #line_dict["gene"] = str( interval [parsing_sample_index - 1] ).split(":")[-1].split("|")[3]
    
    print ( '\t'.join ( list( line_dict.values()) ) )
    print ( '\t'.join ( list( line_dict.values()) ), file = output_file )
   

output_file.close()

## FacetCNV → PycloneVI

In [1]:
import pandas as pd
import gzip
import os, pybedtools

FACETCNV_OUTPUT_PATH="/home/goldpm1/Meningioma/11.cnv/5.facetcnv/220930/Dura/220930.vcf.gz"
FACETCNV_TO_PYCLONEVI_MATRIX_PATH="/home/goldpm1/Meningioma/31.Clonality/01.make_matrix/220930/220930.facetcnv_to_pyclonevi.tsv"
MUTECT_OUTPUT_PATH="/home/goldpm1/Meningioma/04.mutect/03.vep/220930_Dura.MT2.FMC.HF.RMBLACK.vep.vcf"
HC_OUTPUT_PATH="/home/goldpm1/Meningioma/06.hc/01.call/220930/Dura/220930_Dura.vcf"
HC_BLOOD_RANDOM_PICK_PATH="/home/goldpm1/Meningioma/31.Clonality/01.make_matrix/220930/220930.HC.random_pick_50.bed"
TISSUE="Dura"
Sample_ID="220930"

SAMPLE_ID = Sample_ID + "_" + TISSUE



if "gz" in FACETCNV_OUTPUT_PATH:
    input_file = gzip.open (FACETCNV_OUTPUT_PATH, "r")
else:
    input_file = open (FACETCNV_OUTPUT_PATH, "r")
sample_name = []


def parsing (line):
    CHR, POS, REF, ALT = line[0], int(line[1]), line[3], line[4]
    # 7. info
    info_list = line[7].split(';')
    info_dict = {}
    for i in range( len(info_list) ):
        if "=" in info_list[i]:
            info_dict [ info_list[i].split('=')[0] ] = info_list[i].split('=')[1]

    return CHR, POS, REF, ALT, info_list, info_dict



matrix = [] 
colnames = ['CHR','START','END', 'MAJOR_CN', 'MINOR_CN', 'NORMAL_CN', 'TUMOR_PURITY' ]
bed_df = pd.DataFrame (columns = colnames) 

print (FACETCNV_OUTPUT_PATH)

for line in input_file.readlines():
    if "gz" in FACETCNV_OUTPUT_PATH:
	    line = line.decode('utf-8')      # ← gzip file일 경우
    line = line.rstrip("\n")
    
    if line[0] == "#": # Header 저장
        if line[0:8] == "##purity":
            TUMOR_PURITY = line.split("=")[1]
            if TUMOR_PURITY == "NA":
                TUMOR_PURITY = "1.0"
        elif line[0:8] == "##ploidy":
            TOTAL_PLOIDY = line.split("=")[1]
        continue
    
    else:
        line = line.split("\t")
        CHR, POS, REF, ALT , info_list, info_dict = parsing(line)
        START, END = POS, info_dict["END"]
        try:   #"LCN_EM에 . 같은 게 있어서 에러날 때가 있다"
            MAJOR_CN, MINOR_CN =  int(info_dict["TCN_EM"])  - int (info_dict["LCN_EM"]),  int (info_dict["LCN_EM"])
            NORMAL_CN = 2           
        except:
            continue
        output_line = [str(CHR), str(START), str(END), str(MAJOR_CN), str(MINOR_CN), str(NORMAL_CN),  str(TUMOR_PURITY) ]
        output_dict = {}
        for colname_index, colname in enumerate (colnames):
            output_dict[colname] = output_line[colname_index]
            
        bed_df = bed_df.append ( pd.Series( output_dict ), ignore_index = True )


bed_df.tail(15)

bed_df.to_csv (FACETCNV_TO_PYCLONEVI_MATRIX_PATH + ".bed_df", sep = "\t", index = False, header = False)

/home/goldpm1/Meningioma/11.cnv/5.facetcnv/220930/Dura/220930.vcf.gz


In [2]:
bed_df

,CHR,START,END,MAJOR_CN,MINOR_CN,NORMAL_CN,TUMOR_PURITY
0,chr1,65975,53050524,0,0,2,1.0
1,chr1,53062325,112541963,1,1,2,1.0
2,chr1,112550632,115033619,1,0,2,1.0
3,chr1,115033817,149943989,1,1,2,1.0
4,chr1,149944187,151707487,1,0,2,1.0
...,...,...,...,...,...,...,...
85,chr20,6214678,25294209,2,1,2,1.0
86,chr20,25294986,64327986,1,1,2,1.0
87,chr21,5031972,46664375,1,1,2,1.0
88,chr22,10685675,50782266,0,0,2,1.0


In [13]:

######## Mutect call을 골라줌 ###################
bed_pybed_object = pybedtools.BedTool.from_dataframe(bed_df)                   # CNV 정보를 가지고 있는 segment 정보
# mutect_pybed_object = pybedtools.BedTool( MUTECT_OUTPUT_PATH )       # mutect으로 call한 mutation
# a = bed_pybed_object.intersect(mutect_pybed_object, wb = True) 

os.system ("bedtools intersect -a " + MUTECT_OUTPUT_PATH + " -b " + FACETCNV_TO_PYCLONEVI_MATRIX_PATH + ".temp1" +  " -wa -wb > " + FACETCNV_TO_PYCLONEVI_MATRIX_PATH + ".temp2")

input_file = open (FACETCNV_TO_PYCLONEVI_MATRIX_PATH + ".temp2", "r")
output_file = open(FACETCNV_TO_PYCLONEVI_MATRIX_PATH, "w")

print ("########## Mutectt2  ###############" )

for interval in input_file.readlines():
    interval = interval.rstrip("\n").split("\t")
    #print (interval)
    
    line_dict = {}
    
    for index, contents in enumerate( interval ):
        if "GT" in contents:
            parsing_sample_index = index + 2
            break

    line_dict ["mutation_id"] = str(interval [0]) + "_" + str(interval[1])
    line_dict ["sample_id"] = SAMPLE_ID
    line_dict ["ref_counts"] = str( interval[ parsing_sample_index ].split(":")[1].split(",")[0])
    line_dict ["alt_counts"] = str( interval[ parsing_sample_index ].split(":")[1].split(",")[1])
    line_dict["normal_cn"] = str (interval[-2])
    line_dict["major_cn"] = str (interval[-4])
    line_dict["minor_cn"] = str (interval[-3])
    line_dict["tumour_content"] = str( interval[-1])
    line_dict["type"] = "Mutect2"
    
    print ( '\t'.join ( list( line_dict.values()) ) )
    print ( '\t'.join ( list( line_dict.values()) ), file = output_file )


input_file.close()

# os.system ("rm -rf " + FACETCNV_TO_PYCLONEVI_MATRIX_PATH + ".txt")
# os.system ("rm -rf " + FACETCNV_TO_PYCLONEVI_MATRIX_PATH + ".temp1")
# os.system ("rm -rf " + FACETCNV_TO_PYCLONEVI_MATRIX_PATH + ".temp2")
    

########## Mutectt2  ###############
chr1_2787605	220930_Dura	190	5	2	0	0	1.0	Mutect2
chr1_42154829	220930_Dura	186	5	2	0	0	1.0	Mutect2
chr1_44802872	220930_Dura	98	14	2	0	0	1.0	Mutect2
chr1_243221776	220930_Dura	492	48	2	1	1	1.0	Mutect2
chr2_3644016	220930_Dura	830	63	2	1	1	1.0	Mutect2
chr2_44706287	220930_Dura	278	16	2	1	1	1.0	Mutect2
chr2_178554137	220930_Dura	603	19	2	2	2	1.0	Mutect2
chr2_185808060	220930_Dura	113	5	2	2	1	1.0	Mutect2
chr2_222909077	220930_Dura	85	5	2	1	1	1.0	Mutect2
chr2_241240049	220930_Dura	187	5	2	1	1	1.0	Mutect2
chr3_11817144	220930_Dura	363	20	2	1	1	1.0	Mutect2
chr3_49721007	220930_Dura	397	25	2	1	1	1.0	Mutect2
chr5_71656712	220930_Dura	172	8	2	1	1	1.0	Mutect2
chr6_87255842	220930_Dura	257	15	2	1	1	1.0	Mutect2
chr7_106026643	220930_Dura	419	24	2	1	1	1.0	Mutect2
chr7_122386320	220930_Dura	39	4	2	1	1	1.0	Mutect2
chr8_10622949	220930_Dura	168	11	2	1	1	1.0	Mutect2
chr8_81801465	220930_Dura	121	5	2	1	1	1.0	Mutect2
chr8_85126674	220930_Dura	323	21	2	1	1	1.0	Mutect2


In [85]:



######## HC call을 골라줌 ###################
print ("\n########## Haplotype Caller ###############")

hc_blood_pybed_object = pybedtools.BedTool ( HC_BLOOD_RANDOM_PICK_PATH )  
hc_pybed_object = pybedtools.BedTool(HC_OUTPUT_PATH)  #VCF

hc_selected_pybed_object = hc_blood_pybed_object.intersect(hc_pybed_object, wa = True, wb = True)       # Dura/Tumor에서 발견된 HC call 중에서 선택된 50개만 고름 (concordance가 95%이기 때문에 47개정도 예상)


ab = hc_selected_pybed_object.intersect(bed_pybed_object, wa = True, wb = True)


for index, contents in enumerate( ab[0] ):
    if "GT:" in contents:
        parsing_sample_index = index
        break


for interval in ab:
    CHR, POS = interval[0], int (interval[2]) - 1
    line_dict = {}

    print (interval)

    line_dict ["mutation_id"] = str(CHR) + "_"  + str(POS)
    line_dict ["sample_id"] = SAMPLE_ID
    line_dict ["ref_counts"] = str( interval [parsing_sample_index + 2] ).split(":")[1].split(",")[0]
    line_dict ["alt_counts"] = str( interval [parsing_sample_index + 2] ).split(":")[1].split(",")[1]
    line_dict["normal_cn"] = str ( 2 )
    line_dict["major_cn"] = str ( interval[ -4 ] )
    line_dict["minor_cn"] = str ( interval[ -3 ] )
    line_dict["tumour_content"] = str( TUMOR_PURITY  )
    line_dict["type"] = "HC"
    
    print ( '\t'.join ( list( line_dict.values()) ) )
    print ( '\t'.join ( list( line_dict.values()) ), file = output_file )
   

output_file.close()



########## Haplotype Caller ###############
chr1	27549970	27549972	chr1	27549971	rs2076457	C	A	6950.06	.	AC=2;AF=1.00;AN=2;BaseQRankSum=3.244;DB;DP=237;ExcessHet=3.0103;FS=11.577;MLEAC=2;MLEAF=1.00;MQ=60.00;MQRankSum=0.000;QD=29.96;ReadPosRankSum=0.740;SOR=0.782	GT:AD:DP:GQ:PL	1/1:3,229:232:99:6964,659,0	chr1	65975	53050524	0	0	2	1.0

chr1_27549971	220930_Dura	3	229	2	0	2	1.0	HC
chr1	112520502	112520504	chr1	112520503	rs910697	A	G	1822.64	.	AC=1;AF=0.500;AN=2;BaseQRankSum=-2.059;DB;DP=147;ExcessHet=3.0103;FS=3.979;MLEAC=1;MLEAF=0.500;MQ=60.00;MQRankSum=0.000;QD=12.48;ReadPosRankSum=0.962;SOR=0.439	GT:AD:DP:GQ:PL	0/1:66,80:146:99:1830,0,1627	chr1	53062325	112541963	1	1	2	1.0

chr1_112520503	220930_Dura	66	80	2	1	2	1.0	HC
chr1	151709090	151709092	chr1	151709091	rs2280474	G	A	3086.64	.	AC=1;AF=0.500;AN=2;BaseQRankSum=-0.621;DB;DP=259;ExcessHet=3.0103;FS=0.000;MLEAC=1;MLEAF=0.500;MQ=60.00;MQRankSum=0.000;QD=12.30;ReadPosRankSum=0.051;SOR=0.696	GT:AD:DP:GQ:PL	0/1:128,123:251:99:3094,0,3367

In [35]:
print(pybedtools.__version__)

0.9.0
